# CoreScale results for "Revisiting TCP Congestion Control Throughput Models & Fairness Properties at Scale"

## Set up your FABRIC environment


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [3]:
slice_name = "bottleneck-10-test-imc"
n_endpoints = 10

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [4]:
slice = fablib.get_slice(name=slice_name)

In [5]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [6]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Generate flows

### Set experiment parameters

>delay, cca, test_duration, num_servers, flows

delay is the delay to be set at the receiver (20 ms,100 ms,200 ms)

num_servers is the number of ports to be opened on each receiver. For core scale we are opening 10 ports and for edge scale we are opening 1 port

test_duration is the time for which to send the iperf3 flows

cca is the congestion control algorithm (bbr, reno or cubic)

flows is the number of parallel flows to be send from each port

For sending 1000 flows set num_servers=10 and flows=10. This will send 100 flows from each of the 10 senders.

In [18]:
cca="bbr"
delay=100
test_duration=7200
num_servers=10
flows=10

### Get queue statistics on the router before experiment

In [121]:
router_node.execute("tc -p -s -d -j qdisc show dev "+router_egress_name +" >tc_before.txt")

('', '')

In [ ]:
router_node.execute("cat tc_before.txt")

## Remove existing result files from the hosts

In [15]:
for n in receiver_nodes:
    n.execute("rm -f ./*")


for n in sender_nodes:
    n.execute("rm -f ./*")

In [16]:
for n in sender_nodes:
    n.execute("ls")

In [17]:
for n in receiver_nodes:
    n.execute("ls")

### Set delay on the receiver

> Values: 20ms, 100ms, 200ms

Now set up delay on the receiver interface:

First delete any existing queue (don't worry if there is an error, it means there was not!)

In [19]:
for n in receiver_nodes:
    receiver_inf=n.get_interface(network_name= "link-receiver")
    receiver_inf_name = receiver_inf.get_device_name()
    n.execute("sudo tc qdisc del dev " + receiver_inf_name + " root netem")
    n.execute("sudo tc qdisc add dev " + receiver_inf_name + " root netem delay " + str(delay) +"ms limit 1000000")
    

### Start parallel servers on the receivers

In this, the base_port is the starting address of port number

Now start the `iperf3` flows:

In [20]:

        
base_port=50000
for i in range(0,10):
    receiver_nodes[i].execute("sudo killall iperf3")
for i in range(0,10):
    for k in range (1,num_servers+1): 
        
        server_port=base_port+k
        report_file=str(server_port)+"-server.dat"
        receiver_nodes[i].execute_thread("iperf3 -s -p "+str(server_port)+" -D --logfile "+report_file)

In [ ]:
#check files in all receiver
for n in receiver_nodes:
    n.execute("ls")

### Start parallel clients on the senders

In this, the base_port is the starting address of port number on the receiver


In [ ]:
import time
base_port=50000


for i in range (0,10):
    sender_nodes[i].execute("sudo killall iperf3")

for i in range (0,10):
    for k in range (0,num_servers):
        server_port=base_port+k+1
        report_file="sender-10.10.2.1"+str(i)+"-"+str(server_port)+"-"+str(test_duration)+"-"+cca+".txt"
        sender_nodes[i].execute_thread("iperf3 -c 10.10.2.1"+str(i)+ " -p "+str(server_port)+ " -t "+str(test_duration)+ " -C "+cca+" -P "+str(flows)+ " --format k &>"+report_file+ " &")
time.sleep(test_duration+10)              

 iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
 

In [ ]:
#check files on all the senders
for n in sender_nodes:
    n.execute("ls")

### Get queue statistics on the router after experiment

In [ ]:
#get queue statistics after running the experiment
router_node.execute("rm -f tc_after.txt")
router_node.execute("tc -p -s -d -j qdisc show dev " + router_egress_name + " >tc_after.txt")
router_node.execute("cat tc_after.txt")
print(router_egress_name)

## Analyze the results

Transfer files from hosts to router

Calculate sum of bandwidth, square of sum of bandwidth, count of flows and jfi:

In [12]:

sum_bw = []
sum_sq_bw = []
count_flow = []


for n in sender_nodes:
    (sum_sen, serr)=n.execute("grep -r -E \"[0-9].*0.00-"+str(test_duration)+".*sender\" . |awk '{sum+=$7}END {print sum}'")
    sum_bw.append(float(sum_sen.strip()))
    (sum_sq, sqerr)=n.execute("grep -r -E \"[0-9].*0.00-"+str(test_duration)+".*sender\" . |awk '{sum+=$7*$7}END {print sum}'")
    sum_sq_bw.append(float(sum_sq.strip()))
    (ncount, ncerr)=n.execute("grep -r -E \"[0-9].*0.00-"+str(test_duration)+".*sender\" . |awk '{count+=1}END {print count}'")
    count_flow.append(int(ncount.strip()))


tput=sum(sum_bw)
sq=sum(sum_sq_bw)
c=sum(count_flow)

print("Sum of bandwidth is %f Kbits/sec " % tput)

print("Sum of square of bandwidth is %f" % sq)

print("Count of flows is %d" % c)

jfi= (tput*tput)/(c*sq)

print("JFI is %f" % jfi)

814413
6633163379
100
814928
6641894962
100
816368
6665105686
100
821821
6754565589
100
825213
6810145857
100
821423
6747919403
100
825042
6807328402
100
819095
6709344359
100
823735
6785787563
100
824441
6797667201
100
Sum of bandwidth is 8206479.000000 Kbits/sec 
Sum of square of bandwidth is 67352922401.000000
Count of flows is 1000


Calculate packet drop rate

In [14]:
#To get packet dropped:
(drop_before,err_drop_before)=router_node.execute("tail --lines=10 tc_before.txt| grep '\"drops\":' | awk '{print $2}' |cut -d ',' -f1")

#To get packets sent
(sent_before,err_sent_before)=router_node.execute("tail --lines=10 tc_before.txt| grep '\"packets\":' | awk '{print $2}' |cut -d ',' -f1")

#To get packet dropped:
(drop_after,err_drop_after)=router_node.execute("tail --lines=10 tc_after.txt| grep '\"drops\":' | awk '{print $2}' |cut -d ',' -f1")

#To get packets sent
(sent_after, err_sent_after)=router_node.execute("tail --lines=10 tc_after.txt| grep '\"packets\":' | awk '{print $2}' |cut -d ',' -f1")

#Calculate packet drop rate:

dropped=int(drop_after)-int(drop_before)

sent=int(sent_after)-int(sent_before)
drop_rate=float(dropped)/float(sent)

print("packet drop before running experiment: "+ str(drop_before))
print("packet sent before running experiment: " + str(sent_before))
print("packet drop after running experiment: "+ str(drop_after))
print("packet sent after  running experiment: " + str(sent_after))
print("packet drop rate: " + str(drop_rate))


2966582
1039609312
3745651
6137981017
packet drop before running experiment: 2966582

packet sent before running experiment: 1039609312

packet drop after running experiment: 3745651

packet sent after  running experiment: 6137981017

packet drop rate: 0.00015280741481362822
